In [9]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

import matplotlib
matplotlib.use('Agg')

from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np 
import torch
import time

In [10]:
class LeNet(Module):
    def __init__(self,numChannels,classes):
        #call the parent constructor
        super(LeNet,self).__init__()
        
        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = Conv2d(in_channels=numChannels,out_channels=20,
                            kernel_size=(5,5))
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2,2),stride=(2,2))
        
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv2d(in_channels=20,out_channels=50,
                           kernel_size=(5,5))
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2,2),stride=(2,2))
        
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = Linear(in_features=800,out_features=500)
        self.relu3 = ReLU()
        
        # initialize our softmax classifier
        self.fc2 = Linear(in_features=500,out_features=classes)
        self.logSoftmax = LogSoftmax(dim=1)
        
    def forward(self,x):
        # pass the input through our first set of CONV => RELU =>
		# POOL layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        # pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        # flatten the output from the previous layer and pass it
		# through our only set of FC => RELU layers
        x = flatten(x,1)
        x = self.fc1(x)
        x = self.relu3(x)
        
        # pass the output to our softmax classifier to get our output
		# predictions
        x = self.fc2(x)
        output = self.logSoftmax(x)
        
        # return the output predictions
        return output
        

In [11]:
# Hyperparameters
INIT_LR = 1e-3
BATCH_SIZE  = 64
EPOCHS = 10

# train and validation split
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT

# training device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# loading the KMINST dataset
print("[INFO] loading the KMINST dataset.....")

trainData = KMNIST(root='data',train=True,download=True,
                   transform=ToTensor())
testData = KMNIST(root='data',train=False,download=True,
                 transform=ToTensor())

#calculate the train/valid split
print("[INFO] generating the train/validation data split....")
numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
numValSamples = int(len(trainData)* VAL_SPLIT)
(trainData,valData) = random_split(trainData,
        [numTrainSamples,numValSamples],
                                   generator=torch.Generator().manual_seed(42)
                                  )

[INFO] loading the KMINST dataset.....
[INFO] generating the train/validation data split....


In [13]:
# initialize train, validation and test data loader

trainDataLoader = DataLoader(trainData,shuffle=True,
                            batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData,batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testData,batch_size=BATCH_SIZE)

#calc the steps per epoch for training and validation step
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [14]:
#intialize the LeNet Model
print("[INFO] Intializing the LeNet model.....")
model = LeNet(
    numChannels=1,
    classes=len(trainData.dataset.classes)).to(device)

# initialize optimizer and loss fnx
opt = Adam(model.parameters(),lr=INIT_LR)
lossFn = nn.NLLLoss()

# intialize history to store a dictionary to store training history
H = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

# measure how long training will take
print("[INFO] training network ....")
startTime = time.time()

[INFO] Intializing the LeNet model.....
[INFO] training network ....


In [19]:
for epoch in range(0,EPOCHS):
    # intialize model training,train loss and validation loss
    model.train()
    
    totalTrainLoss = 0
    totalValLoss = 0
    
    # intialize n0. of correct predictions in train and val step
    trainCorrect = 0
    valCorrect = 0
    
    # loop over training set
    for (x,y) in trainDataLoader:
        
        #send input data to device
        (x,y) = (x.to(device),y.to(device))
        
        # forward pass and calc train loss
        pred = model(x)
        loss = lossFn(pred,y)
        
        # zero the gradients, perform backpropagation and update weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        #compute total loss and calc num of correct predictions
        totalTrainLoss += loss
        trainCorrect += (pred.argmax(1)==y).type(
        torch.float).sum().item()
    
    
    
    # turn off autograd for evaluation
    with torch.no_grad():
        # intialize model evaluation
        model.eval()
        
        # loop over validation set
        for (x,y) in valDataLoader:
            #transfer data to device
            (x,y) = (x.to(device),y.to(device))
            
            # make predictions and compute loss
            pred = model(x)
            totalValLoss += lossFn(pred,y)
            
            # calc the num of correct predictions
            valCorrect += (pred.argmax(1)==y).type(
            torch.float).sum().item()
            
    # average train and validation loss        
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps
    
    # calc training and validation accuracy
    trainCorrect = trainCorrect / len(trainDataLoader.dataset)
    valCorrect = valCorrect / len(valDataLoader.dataset)
    
    # update trianing history
    H['train_loss'].append(avgTrainLoss.cpu().detach().numpy)
    H['train_acc'].append(trainCorrect)
    H['val_loss'].append(avgValLoss.cpu().detach().numpy())
    H['val_acc'].append(valCorrect)
    
    # print the model trianing and validation information
    
    print("[INFO] EPOCH: {}/{}".format(epoch+1,EPOCHS))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
    avgTrainLoss,trainCorrect))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
    avgValLoss,valCorrect))
    
    # finish measuring how long training took
    endTime = time.time()
    print("[INFO] total time taken to train the model: {:.2f}s".format(
        endTime - startTime))
    # we can now evaluate the network on the test set
    print("[INFO] evaluating network...")
    # turn off autograd for testing evaluation
    with torch.no_grad():
        # set the model in evaluation mode
        model.eval()

        # initialize a list to store our predictions
        preds = []
        # loop over the test set
        for (x, y) in testDataLoader:
            # send the input to the device
            x = x.to(device)
            # make the predictions and add them to the list
            pred = model(x)
            preds.extend(pred.argmax(axis=1).cpu().numpy())
    # generate a classification report
    print(classification_report(testData.targets.cpu().numpy(),
        np.array(preds), target_names=testData.classes))
            
    
            

[INFO] EPOCH: 1/10
Train loss: 0.057091, Train accuracy: 0.9824
Val loss: 0.085069, Val accuracy: 0.9751

[INFO] total time taken to train the model: 738.18s
[INFO] evaluating network...
              precision    recall  f1-score   support

           o       0.97      0.89      0.93      1000
          ki       0.96      0.93      0.95      1000
          su       0.92      0.88      0.90      1000
         tsu       0.96      0.93      0.94      1000
          na       0.93      0.90      0.92      1000
          ha       0.95      0.95      0.95      1000
          ma       0.90      0.96      0.93      1000
          ya       0.95      0.94      0.95      1000
          re       0.87      0.99      0.92      1000
          wo       0.95      0.96      0.96      1000

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000

[INFO] EPOCH: 2/10
Train loss: 0.037549, Train accurac

[INFO] EPOCH: 10/10
Train loss: 0.009393, Train accuracy: 0.9970
Val loss: 0.096164, Val accuracy: 0.9797

[INFO] total time taken to train the model: 1867.96s
[INFO] evaluating network...
              precision    recall  f1-score   support

           o       0.95      0.96      0.96      1000
          ki       0.98      0.92      0.95      1000
          su       0.87      0.92      0.90      1000
         tsu       0.95      0.97      0.96      1000
          na       0.93      0.94      0.93      1000
          ha       0.96      0.92      0.94      1000
          ma       0.93      0.96      0.94      1000
          ya       0.98      0.95      0.96      1000
          re       0.92      0.99      0.95      1000
          wo       0.98      0.93      0.96      1000

    accuracy                           0.94     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.94      0.95     10000

